In [1]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [2]:
from sklearn.metrics import mean_squared_log_error

In [3]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
train=pd.read_csv("C://Users//Shashank//Downloads//Open_cv_Assingment//DataSets//train.csv")
test=pd.read_csv("C://Users//Shashank//Downloads//Open_cv_Assingment//DataSets//test.csv")

In [4]:

y=train['traffic_volume']
del train['traffic_volume']

In [5]:
categorical_train=train.select_dtypes(include=[object])
categorical_test=test.select_dtypes(include=[object])

In [6]:
categorical_train.head()

,date_time,is_holiday,weather_type,weather_description
0,2012-10-02 09:00:00,None,Clouds,scattered clouds
1,2012-10-02 10:00:00,None,Clouds,broken clouds
2,2012-10-02 11:00:00,None,Clouds,overcast clouds
3,2012-10-02 12:00:00,None,Clouds,overcast clouds
4,2012-10-02 13:00:00,None,Clouds,broken clouds


In [7]:
#del the date_time
del categorical_train['date_time']
del categorical_test['date_time']

In [8]:
enc = OneHotEncoder(handle_unknown='ignore',sparse=True)
categorical_train_trans=enc.fit_transform(categorical_train)
categorical_test_trans=enc.transform(categorical_test)

In [9]:
#droping the columns in the orginal train and test for which we have done one-hot encoding
train = train.drop(["is_holiday", "weather_type", "weather_description"], axis=1)
test = test.drop(["is_holiday", "weather_type", "weather_description"], axis=1)

In [10]:
train_df = pd.DataFrame(categorical_train_trans.toarray())
test_df=pd.DataFrame(categorical_test_trans.toarray())

In [11]:
train = pd.concat([train, train_df], axis=1) 
test=pd.concat([test, test_df], axis=1) 

In [12]:
print(train.shape)
print(test.shape)

(33750, 72)
(14454, 72)


In [13]:
def make_date(df, date_field:str):
    "Make sure `df[field_name]` is of the right date type."
    field_dtype = df[date_field].dtype
    if isinstance(field_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        field_dtype = np.datetime64
    if not np.issubdtype(field_dtype, np.datetime64):
        df[date_field] = pd.to_datetime(df[date_field], infer_datetime_format=True)

In [14]:
def ifnone(a,b):
    "`a` if `a` is not None, otherwise `b`."
    return b if a is None else a

In [15]:
import re
def add_datepart(df, field_name:str, prefix:str=None, drop:bool=True, time:bool=False):
    "Helper function that adds columns relevant to a date in the column `field_name` of `df`."
    make_date(df, field_name)
    field = df[field_name]
    prefix = ifnone(prefix, re.sub('[Dd]ate$', '', field_name))
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear', 'Is_month_end', 'Is_month_start', 
            'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[prefix + n] = getattr(field.dt, n.lower())
    df[prefix + 'Elapsed'] = field.astype(np.int64) // 10 ** 9
    if drop: df.drop(field_name, axis=1, inplace=True)
    return df

In [16]:
import numpy as np
add_datepart(train, 'date_time',time=True) # inplace
add_datepart(test, 'date_time',time=True)
train.head()

,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,...,date_timeIs_month_end,date_timeIs_month_start,date_timeIs_quarter_end,date_timeIs_quarter_start,date_timeIs_year_end,date_timeIs_year_start,date_timeHour,date_timeMinute,date_timeSecond,date_timeElapsed
0,121,89,2,329,1,1,288.28,0.0,0.0,40,...,False,False,False,False,False,False,9,0,0,1349168400
1,178,67,3,330,1,1,289.36,0.0,0.0,75,...,False,False,False,False,False,False,10,0,0,1349172000
2,113,66,3,329,2,2,289.58,0.0,0.0,90,...,False,False,False,False,False,False,11,0,0,1349175600
3,20,66,3,329,5,5,290.13,0.0,0.0,90,...,False,False,False,False,False,False,12,0,0,1349179200
4,281,65,3,329,7,7,291.14,0.0,0.0,75,...,False,False,False,False,False,False,13,0,0,1349182800


In [17]:
#train
del train['date_timeMinute']
del train['date_timeSecond']
#test
del test['date_timeMinute']
del test['date_timeSecond']

In [18]:
print(train.shape)
print(test.shape)

(33750, 85)
(14454, 85)


In [19]:
x_train=train[0:27750]
y_train=y[0:27750]
x_valid=train[27750:33750]
y_valid=y[27750:33750]

In [47]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
param_test ={'num_leaves': [3,5,7,9,11], 
             'min_child_samples': [2,3,5], 
             'min_child_weight': [ 1e-2, 1e-1, 1, 1e1, 1e2],
             'subsample': [0.3,0.5,0.7], 
             'colsample_bytree': [0.2,0.3,0.5],
             'reg_alpha': [5, 7, 10, 50],
             'reg_lambda': [10, 20, 100],
             'max_depth':[60,65,70,75],
             'n_estimators':[3000,4000,5000]
            }

In [48]:
#This parameter defines the number of HP points to be tested
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
n_HP_points_to_test = 100

import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
tscv = TimeSeriesSplit(n_splits=2)
#n_estimators is set to a "large value". The actual number of trees build will depend on early stopping and 5000 define only the absolute maximum
clf = lgb.LGBMRegressor( random_state=314, silent=True, metric='None', n_jobs=-1)
gs = RandomizedSearchCV(
    estimator=clf, param_distributions=param_test, 
    n_iter=n_HP_points_to_test,
    scoring='neg_mean_squared_error',
    cv=tscv,
    refit=True,
    random_state=314,
    verbose=True)

In [49]:
%%time
gs.fit(x_train, y_train)
print('Best score reached: {} with params: {} '.format(gs.best_score_, gs.best_params_))

Fitting 2 folds for each of 100 candidates, totalling 200 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed: 25.6min finished


Best score reached: -311918.57135259814 with params: {'subsample': 0.3, 'reg_lambda': 100, 'reg_alpha': 7, 'num_leaves': 11, 'n_estimators': 3000, 'min_child_weight': 0.1, 'min_child_samples': 5, 'max_depth': 70, 'colsample_bytree': 0.2} 
Wall time: 25min 43s


In [50]:
gs.best_params_

{'subsample': 0.3,
 'reg_lambda': 100,
 'reg_alpha': 7,
 'num_leaves': 11,
 'n_estimators': 3000,
 'min_child_weight': 0.1,
 'min_child_samples': 5,
 'max_depth': 70,
 'colsample_bytree': 0.2}

In [51]:
from sklearn.metrics import mean_squared_error
y_hat=gs.best_estimator_.predict(x_valid)
y_pred=y_hat.astype(int)
np.sqrt(mean_squared_error( y_valid, y_pred ))

567.1085938042789

In [20]:
#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(x_train.values)
    rmse= np.sqrt(-cross_val_score(model, x_train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

# Base Models

In [21]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

In [22]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

In [23]:
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)


In [24]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

In [25]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

In [26]:
from sklearn.ensemble import RandomForestRegressor
model_rf=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=68,
           max_features=25, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=4,
           min_samples_split=3, min_weight_fraction_leaf=0.0,
           n_estimators=615, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)


# Average Stacking Models

In [27]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)   

In [ ]:
averaged_models = AveragingModels(models = (ENet, GBoost, KRR, lasso,model_xgb,model_rf))

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

C:\Users\Shashank\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\Shashank\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:167: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "


# Stacking Models

In [22]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
          # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)      

In [ ]:
ensemble = stacked_pred*0.70 + xgb_pred*0.15 + lgb_pred*0.15

In [ ]:
from sklearn.ensemble import RandomForestRegressor
clf=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=68,
           max_features=25, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=4,
           min_samples_split=3, min_weight_fraction_leaf=0.0,
           n_estimators=615, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)


In [20]:
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
# Various hyper-parameters to tune
xgb1 = XGBRegressor(njobs=-1)
parameters = {
              'learning_rate': [0.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [1,5,8],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [300,500]}

xgb_grid = GridSearchCV(xgb1,parameters,cv = 2,n_jobs = 5,verbose=True)

xgb_grid.fit(x_train,y_train)


Fitting 2 folds for each of 54 candidates, totalling 108 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.


KeyboardInterrupt: 

In [ ]:
print(xgb_grid.best_params_)

In [ ]:
xgb2= XGBRegressor(colsample_bytree=0.7,learning_rate=0.03,max_depth=5,min_child_weight=5,n_estimators=300,subsample=0.7)
xgb2.fit(x_train,y_train)

In [ ]:
y_hat=xgb2.predict(x_valid)
y_pred=y_hat.astype(int)
y_pred

In [ ]:
from sklearn.metrics import mean_squared_error
aa=mean_squared_error(y_valid, y_pred)
rmse=np.sqrt(aa)
rmse

In [ ]:
y_hat_test=xgb2.predict(test)
y_pred=y_hat_test.astype(int)
y_pred

In [ ]:
test=pd.read_csv("C://Users//sahana//Downloads//hackerearth//8f504160af7a11e9//DataSets//test.csv")
date_time=test['date_time']

In [ ]:
sub = pd.DataFrame()
sub['date_time'] = date_time
sub['traffic_volume'] = y_pred
sub.to_csv('xgb.csv',index=False)

In [ ]:
sub.head()

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
xgb1 = XGBRegressor(n_jobs=-1)
tscv = TimeSeriesSplit(n_splits=2)
parameters = {
              'learning_rate': [0.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [1,5,8],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [300,500]}

xgb_grid = GridSearchCV(xgb1,parameters,cv = tscv,n_jobs = 5,verbose=True,scoring='neg_mean_squared_log_error')

xgb_grid.fit(x_train,y_train)

In [ ]:
xgb_grid.best_estimator_

In [ ]:
y_hat=xgb_grid.best_estimator_.predict(x_valid)
y_pred=y_hat.astype(int)
np.sqrt(mean_squared_error(y_valid, y_pred ))

In [ ]:
y_hat=xgb_grid.best_estimator_.predict(x_valid)
y_pred=y_hat.astype(int)
np.sqrt(mean_squared_log_error(y_valid, y_pred ))

In [ ]:
type(y_pred)

In [ ]:
len(y_valid)

In [ ]:
np.isnan(y_pred).any()

In [ ]:
train.columns